In [ ]:
!pip install mesa

In [ ]:
import mesa
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import random
import numpy as np

#Attempt 0

In [ ]:
"""class SchellingAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.type = np.random.choice([0, 1])  # Tipo de agente: 0 o 1

    def step(self):
        similar_neighbors = sum([1 for neighbor in self.model.grid.neighbor_iter(self.pos) if neighbor.type == self.type])
        if similar_neighbors < self.model.homophily:
            self.model.grid.move_to_empty(self)
        else:
            self.model.satisfied += 1

class SchellingModel(Model):
    def __init__(self, width, height, density, homophily):
        self.width = width
        self.height = height
        self.density = density
        self.homophily = homophily
        self.grid = SingleGrid(width, height, torus=True)
        self.schedule = RandomActivation(self)
        self.satisfied = 0

        # Crear agentes
        self.create_agents()

        # Recolector de datos
        self.datacollector = DataCollector(
            agent_reporters={"Type": "type"}
        )

    def create_agents(self):
        for (content, x, y) in self.grid.coord_iter():
            if self.random.random() < self.density:
                agent = SchellingAgent((x, y), self)
                self.grid.position_agent(agent, (x, y))
                self.schedule.add(agent)

    def step(self):
        self.satisfied = 0
        self.schedule.step()
        self.datacollector.collect(self)

# Parámetros del modelo
width = 20
height = 20
density = 0.8
homophily = 3

# Crear modelo
model = SchellingModel(width, height, density, homophily)

# Ejecutar la simulación
for i in range(100):
    model.step()

# Obtener los resultados
agent_counts = np.zeros((model.width, model.height))
for cell in model.grid.coord_iter():
    cell_content, x, y = cell
    agent_count = len(cell_content)
    agent_counts[x][y] = agent_count

print("Agent Counts:")
print(agent_counts)
"""

/usr/local/lib/python3.10/dist-packages/mesa/time.py:82: FutureWarning: The AgentSet is experimental. It may be changed or removed in any and all future releases, including patch releases.
We would love to hear what you think about this new feature. If you have any thoughts, share them with us here: https://github.com/projectmesa/mesa/discussions/1919
  self._agents: AgentSet = AgentSet(agents, model)


ValueError: not enough values to unpack (expected 3, got 2)

#Attempt 1

In [ ]:
class VehicleAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.route = []  # Ruta del vehículo

    def move(self):
        # Algoritmo de enrutamiento (Algoritmo Genético)
        if not self.route:
            self.route = self.genetic_algorithm()

        # Mover el vehículo a la siguiente posición en la ruta
        if self.route:
            next_position = self.route.pop(0)
            self.model.grid.move_agent(self, next_position)

    def genetic_algorithm(self):
        # Implementar aquí el algoritmo genético para encontrar la ruta óptima
        # Esta es una implementación de ejemplo que genera una ruta aleatoria
        route_length = self.model.grid.width + self.model.grid.height - 2
        route = [(self.pos[0], y) for y in range(1, self.model.grid.height)] + [(x, self.model.grid.height - 1) for x in range(1, self.model.grid.width)]
        random.shuffle(route)
        return route

class RoutingModel(Model):
    def __init__(self, num_agents, width, height):
        self.num_agents = num_agents
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)

        # Crear agentes vehiculares
        for i in range(self.num_agents):
            a = VehicleAgent(i, self)
            self.schedule.add(a)
            # Añadir agente a la grilla en una ubicación aleatoria
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))

        self.datacollector = DataCollector(agent_reporters={"Route": "route"})

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

# Función para calcular la entropía del sistema
def calculate_entropy(model):
    # Lógica para calcular la entropía del sistema
    # Obtener datos sobre las rutas de los vehículos
    routes = model.datacollector.get_agent_vars_dataframe()
    routes = routes.droplevel(0)  # Eliminar el índice de tiempo

    # Calcular la frecuencia de cada ruta
    route_counts = routes['Route'].value_counts()
    total_routes = len(routes)

    # Calcular la probabilidad de cada ruta
    probabilities = route_counts / total_routes

    # Calcular la entropía de Shannon
    entropy = -np.sum(probabilities * np.log2(probabilities))

    return entropy

In [ ]:
# Parámetros de la simulación
num_agents = 50
grid_width = 10
grid_height = 10
num_steps = 100

# Inicializar modelo
model = RoutingModel(num_agents, grid_width, grid_height)

# Ejecutar simulación
for i in range(num_steps):
    model.step()

# Calcular entropía del sistema
entropy = calculate_entropy(model)
print("Entropía del sistema:", entropy)

<ipython-input-8-c1bfe906b025>:3: FutureWarning: The Mesa Model class was not initialized. In the future, you need to explicitly initialize the Model by calling super().__init__() on initialization.
  super().__init__(unique_id, model)


AttributeError: 'RoutingModel' object has no attribute '_steps'

#Attempt 2

In [ ]:
import random
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import NetworkGrid
from mesa.datacollection import DataCollector
import networkx as nx

In [ ]:
class OpinionAgent(Agent):
    """An agent with an opinion."""
    def __init__(self, unique_id, model, initial_opinion):
        super().__init__(unique_id, model)
        self.opinion = initial_opinion

    def step(self):
        # Choose a random neighbor
        if self.opinion == 'A':
            neighbors = self.model.grid.get_neighbors(self.pos, include_center=False)
            if neighbors:
                other_agent = random.choice(neighbors)
                # Propagate opinion
                other_agent.opinion = 'A'
        else:
            # Do nothing if already has opinion B
            pass

class OpinionModel(Model):
    """A model with some number of agents."""
    def __init__(self, num_agents, avg_node_degree):
        super().__init__()  # Llamamos a la inicialización del modelo
        self.num_agents = num_agents
        self.network = nx.erdos_renyi_graph(num_agents, avg_node_degree / num_agents)
        self.grid = NetworkGrid(self.network)
        self.schedule = RandomActivation(self)

        # Create agents
        for i, node in enumerate(self.network.nodes()):
            initial_opinion = 'A' if random.random() < 0.5 else 'B'
            a = OpinionAgent(i, self, initial_opinion)
            self.schedule.add(a)
            # Add agent to grid
            self.grid.place_agent(a, node)

        # Data collector to track opinion distribution
        self.datacollector = DataCollector(
            agent_reporters={"Opinion": "opinion"}
        )

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

In [ ]:
def cal_entropia(data):
  import numpy as np

  # Obtener la distribución de opiniones de los agentes en el último paso de la simulación
  opinions = data.loc[99, 'Opinion']

  # Contar la cantidad de agentes con cada opinión
  opinion_counts = opinions.value_counts()

  # Calcular la probabilidad de cada opinión
  probabilities = opinion_counts / opinion_counts.sum()

  # Calcular la entropía de Shannon
  entropy = -np.sum(probabilities * np.log2(probabilities))

  # Redondear la entropía a 5 decimales
  entropy = round(entropy, 5)

  return entropy

In [ ]:
# Parameters
num_agents = 50
avg_node_degree = 3

# Create model
model = OpinionModel(num_agents, avg_node_degree)

# Run the model
for i in range(100):
    model.step()

# Retrieve data
data = model.datacollector.get_agent_vars_dataframe()
print(data)

             Opinion
Step AgentID        
0    0             A
     1             A
     2             B
     3             B
     4             B
...              ...
99   37            A
     16            A
     9             A
     20            A
     4             A

[5000 rows x 1 columns]


In [ ]:
entropy = cal_entropia(data)
print("Entropía de la distribución de opiniones:", entropy)

Entropía de la distribución de opiniones: 0.14144


https://mesa.readthedocs.io/en/stable/tutorials/intro_tutorial.html